In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from dataloader.dataloader import DataLoader
from preprocessing.clustering_segmentation import ClusteringSegmentation
from preprocessing.edge_detection import EdgeDetection
from preprocessing.image_restoration import ImageRestorer
from preprocessing.preproccessing import ImagePreprocessor
from preprocessing.region_segmentation import RegionBasedSegmentation
from preprocessing.threshold_segmentation import ThresholdSegmentation
from feature_extraction.feature_extraction import FeatureExtractor
from feature_selection.feature_selection import FeatureSelector
from model_selection.model_selection import ModelSelection
from performance_analysis.performance_analysis import PerformanceAnalysis

import matplotlib.pyplot as plt
from skimage.color import rgb2gray
import numpy as np

## Data Loading

In [ ]:
dataloader = DataLoader(Path('./data'))

In [26]:
# Load Data from files
x_train, y_train, x_test, y_test, x_val, y_val = dataloader.load_data()

In [ ]:
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")
print(f"x_val shape: {x_val.shape}")
print(f"y_val shape: {y_val.shape}")

In [ ]:
plt.imshow(x_train[100])
plt.show()

## Preprocessing

In [ ]:
clustering_segmentation = ClusteringSegmentation(method='kmeans', n_clusters=3, compactness=30.0, sigma=1.0)
plt.imshow(clustering_segmentation.process(x_train[0]))
plt.show()

In [ ]:
edge_detection = EdgeDetection(method='sobel')
#method: for roberts and canny: image must be 2D
plt.imshow(edge_detection.process(x_train[0]))
plt.show()

In [ ]:
image_restorer = ImageRestorer(method='median')
#'mean_rectangular', 'gaussian', 'adaptive' and 'wiener' generate an error
plt.imshow(image_restorer.process(x_train[0]))
plt.show()

In [ ]:
image_preprocessor = ImagePreprocessor(method = 'CLAHE')
# method = 'log' generates an error
plt.imshow(image_preprocessor.process(x_train[0]))
plt.show()

In [ ]:
region_based_segmentation = RegionBasedSegmentation(method='region_merging')
# method: 'region_splitting' generates an error
plt.imshow(region_based_segmentation.process(x_train[0]))
plt.show()

## Features Extraction

In [ ]:
feature_extractor = FeatureExtractor()

# HOG FEATURES
hog_features = []
#10 images with label = 0
for i in range(10):
    hog_features.append(feature_extractor.extract_hog_features(x_train[i]))
# 10 images with label = 3
for i in range(1000, 1010):
    hog_features.append(feature_extractor.extract_hog_features(x_train[i]))
hog_features = np.array(hog_features)
print(f"hog_features shape: {hog_features.shape}")

# LCP FEATURES
lbp_features = []
for i in range(10):
    lbp_features.append(feature_extractor.extract_lbp_features(rgb2gray(x_train[i])))
lbp_features = np.array(lbp_features)
print(f"lbp_features shape: {lbp_features.shape}")


# SIFT FEATURES (error)
# sift_features = []
# for i in range(10):
#     sift_features.append(feature_extractor.extract_sift_features(X_train[i]))
# sift_features = np.array(sift_features)
# print(f"sift_features shape: {sift_features.shape}")

# FOURIER DESCRIPTOR FEATURES (error)
# fourier_descriptor_features = []
# for i in range(10):
#     fourier_descriptor_features.append(feature_extractor.extract_fourier_descriptor_features(X_train[i]))
# fourier_descriptor_features = np.array(fourier_descriptor_features)
# print(f"fourier_descriptor_features shape: {fourier_descriptor_features.shape}")

# SURF FEATURES (error)

surf_features = []
# for i in range(10):
#     surf_features.append(feature_extractor.extract_surf_features(X_train[i]))
# surf_features = np.array(surf_features)
# print(f"surf_features shape: {surf_features.shape}")

## Feature Selection

In [ ]:
feature_selector = FeatureSelector()


# PCA FEATURES
pca_features = []
for i in range(10):
    pca_features.append(feature_selector.extract_pca_features(x_train[i]))
pca_features = np.array(pca_features)
print(f"pca_features shape: {pca_features.shape}")

## Model Selection

In [ ]:
# true_labels is just for trial here
true_labels = np.concatenate((np.zeros(10), np.ones(10) * 3))
model_selection = ModelSelection(
    hog_features[:15], true_labels[:15], hog_features[15:], true_labels[15:]
)
knn, pred_train, pred_val = model_selection.KNN()
print(pred_train)


In [ ]:
performance_analysis = PerformanceAnalysis('KNN', pred_train, true_labels[15:])
performance_analysis.calculate_performance_metrics()